# Preliminaries

Fix openai version to version 0.28. They did a major overhaul on the python code base and it breaks a lot of stuff. Will update once langchain and others catch up. See https://github.com/openai/openai-python for details.

In [1]:
# pip install openai==0.28

Install all of the required packages with the requirements.txt file provided

In [ ]:
# pip install -r requirements.txt

## Langsmith

* Langsmith is cool. It automatically logs and tags every single code run you do. 
* Link to Langsmith, code being logged under the Aerospace AI project. https://smith.langchain.com/o/45eb8917-7353-4296-978d-bb461fc45c65 

# Setup

In [2]:
"""
Page setup and execution for the aerospace mechanism chatbot
Example :        
-What can you tell me about latch mechanism design failures which have occurred        
-Follow up: Which one of the sources discussed volatile spherical joint interfaces           
"""
# import databutton as db

import os
import glob
import data_import
import queries
from dotenv import load_dotenv, find_dotenv
import streamlit as st
import openai
import secrets

from IPython.display import display, Markdown
import pprint

from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

In [3]:
# Check api keys
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))
# print(os.getenv('HUGGING_FACE_API_KEY'))
# print(os.getenv('LANGCHAIN_TRACING_V2'))
# print(os.getenv('LANGCHAIN_ENDPOINT'))
# print(os.getenv('LANGCHAIN_API_KEY'))
# print(os.getenv('LANGCHAIN_PROJECT'))

## Tradeable fields

In [4]:
output_level=None   # Max token limit, see section below
k=4 # Number of queries to return
search_type='similarity'    #  'mmr' or 'similarity'
temperature=0
verbose=True
chain_type='stuff'  # 'stuff' or  'map_reduce'

In [5]:
# Process some items
if output_level=="Concise":
    max_generated_tokens=50
elif output_level=="Detailed":
    max_generated_tokens=516
else:
    max_generated_tokens = -1
    # max_generated_tokens=None # Openai>0.28 requires this, not -1.

# Track filtering, set history to be blank
message_id=0
filter_toggle=False # Filter sources on last answer
chat_history = []

## Vector database

### Pinecone

In [6]:
import pinecone

pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENVIRONMENT')
)
index_name = 'canopy--ams'

## Embeddings

### Openai Embeddings

In [7]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

### Voyage embeddings

## LLM

Run one of the cells below to get an llm that plugs into the prompts section.

### Proprietary

#### Openai

* Models: https://platform.openai.com/docs/models/gpt-3-5, https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo
* This code will break if openai>0.28. See https://github.com/openai/openai-python for details. Langchain yells at you but it still works for now.

In [41]:
from langchain.llms import OpenAI

openai.api_key = os.getenv('OPENAI_API_KEY')

# model_name='gpt-3.5-turbo-instruct'
model_name='gpt-3.5-turbo-1106' # 16,385 tokens
# model_name='gpt-4-0613'

llm = OpenAI(temperature=temperature,
             model_name=model_name,
             max_tokens=max_generated_tokens,
             tags=[model_name+'-'+str(temperature)+'-'+str(max_generated_tokens)])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain/llms/openai.py:244: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain/llms/openai.py:1043: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [42]:
llm("What year was albert einstein born?")

'Albert Einstein was born on March 14, 1879.'

####  Anthropic, Claude

### Open Source

#### Hosted

##### Hugging face models (via API)

* Choose a model_name from the leaderboard here: https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard 
* When testing these, many of the models will time out. 

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

model_name='google/flan-t5-xxl'   # Fast but with uninteresting answers
# model_name='tiiuae/falcon-40b'  # Timed out
# model_name='WizardLM/WizardLM-70B-V1.0'   # Timed out
# model_name='HuggingFaceH4/zephyr-7b-beta' # Timed out
# model_name='meta-llama/Llama-2-70b-chat-hf'   # Gated repo access, request open

llm = HuggingFaceHub(repo_id=model_name,
                     model_kwargs={"temperature": 0.1, "max_length": 250}
)

question = "Who won the FIFA World Cup in the year 1994? "
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(question))

In [ ]:
text="""
Generate a question that a human would ask to get the following response from a chatbot who is knowledgable about aerospace mechanisms:
Dynamic Behavior of Ball Bearings under Axial Vibration  Virgil Hinque* and René Seiler* Abstract  The paper addresses the dynamics of ball bearings when exposed to vibration loads along their axis of  rotation. Following common practice in space mechanisms design, the bearings are mounted in either hard  preloaded or soft preloaded pairs. A computer-based model has been developed for the analysis and  prediction of the load-deflection characteristics in bearing systems. Furthermore, the model may be used  to quantify the maximum loads applied on the bearings and the resulting stresses during a vibration test or  a spacecraft launch.  In parallel to the model development, an experimental test program has been carried out in order to get  sufficient data for model correlation. In this context, the paper also elaborates on the post-processing of the  acquired test signals and discusses specific effects, for instance nonlinearities due to the use of snubbers,  in the time domain as well as in the frequency domain.  Introduction  Many space mechanisms use ball bearings for rotation functions. Therefore, assessing the bearing  performance for the relevant environmental conditions is one of the typical challenges faced during the  equipment design process. In this frame, it is common engineering practice to reduce the effect of a sine  and random vibration environment to quasi-static equivalent loads and stresses. The relevant ball bearing  systems often comprise two identical deep-groove or angular-contact bearings in an axially preloaded  configuration. Several studies on the influence of the preload and other parameters on the structural  behavior of such bearing assemblies have been done by the European Space Tribology Laboratory (ESTL).  In a recent investigation, 25 ball bearing cartridges (“test units” or “bearing housings”) with different preload  and snubber configurations were submitted to a series of sine and random vibration tests. The discussion  of findings was mainly based on the analysis of frequency-domain data and bearing damage assessment  via visual inspection [1].  The ESTL investigation inspired a number of ideas for continuation of the research, among others the  development of a computer-based model that would be able to simulate the behavior of the bearing  cartridges, especially those showing nonlinear features in their response. An adequate model should be  able to predict the load transmission across the bearings in static and dynamic load situations. As the main  sizing criterion for ball bearings is based on the allowable peak Hertzian contact pressure between the balls  and the races [2], accurate knowledge of the maximum bearing loads is a key aspect for successful bearing  selection and implementation in a space mechanism.  During the current investigation at the European Space Research and Technology Centre (ESTEC), a  model was built using MATLAB®/Simulink®, with only the axial degree of freedom in a bearing taken into  consideration. Because model correlation with real test results is of importance, a test program  complementary to that reported in [1] has been conducted, with specific focus on the acquisition and  interpretation of time-domain data. The following chapters describe the computer-based model, the design  of the test units, as well as the details of the test campaign and corresponding results. The last part of the  paper is dedicated to the comparison between the model output and the experimental test data.  *European Space Agency (ESA/ESTEC), Noordwijk, The Netherlands Proceedings of the 44th Aerospace Mechanisms Symposium, NASA Glenn Research Center, May 16-18, 2018 NASA/CP—2018-219887 83
"""
print(llm(text))

### Local

##### Ollama (local, works on mac only)

To get to work, download ollama and install on your mac. This tended to crash my computer (defaults to running on GPU, no option to modify). Use at risk.

In [ ]:
# from langchain.llms import Ollama

# model_name='llama2:latest'
# llm = Ollama(model=model_name)

In [ ]:
# llm("the first man on the moon was...")

#### Local model hosted with LM Studio

This is awesome. Install https://lmstudio.ai/
* Download a model. I installed Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q6_K.gguf and mounted it on my macbook pro. 16gb of RAM, can get around 6000 word input responses in around 10 seconds.
* Start a server locally and run the code below. This will not work unless you have the openai version 0.28 installed.

In [38]:
from langchain.llms import OpenAI

openai.api_key = os.getenv('OPENAI_API_KEY')

# model_name='local-model'    # This can be anything. I recommend using the name of the loaded model from LM Studio.
# model_name='TheBloke/Mistral-7B-Instruct-v0.1-GGUF'
model_name='TheBloke/Llama-2-7B-Chat-GGUF'

base_url='http://localhost:1234/v1' # point to the local server
max_tokens=256  # Set this differently because these models have different restrictions than OpenAI

llm = OpenAI(model_name=model_name,
             base_url=base_url,
             temperature=temperature,
             max_tokens=256,
             tags=[model_name+'-'+str(temperature)+'-'+str(max_generated_tokens)]
            )
             # temperature=temperature,
             # max_tokens=max_generated_tokens,
             # tags=[model_name+'-'+str(temperature)+'-'+str(max_generated_tokens)])

In [39]:
llm("What year was albert einstein born?")

'  Albert Einstein was born on March 14, 1879, in Ulm, Kingdom of Württemberg, German Empire.'

# Prompts

In [25]:
prompt=['Prompt 1: What types of lubricants are to be avoided when designing space mechanisms?',
        'Prompt 2: Can you speak to what failures have occurred when using mineral oil lubricants?']

## First prompt

In [26]:
# Set up object
tags=['prompt1']
qa_model_obj=queries.QA_Model(index_name,
                    embeddings_model,
                    llm,
                    k,
                    search_type,
                    verbose,
                    filter_arg=filter_toggle)

# Generate a response using your chat model
qa_model_obj.query_docs(prompt[0],tags=tags)
ai_response=qa_model_obj.result['answer']
references=qa_model_obj.sources[-1]

display(Markdown(prompt[0]))
display(Markdown(ai_response))
pprint.pprint(references,width=500)

Prompt 1: What types of lubricants are to be avoided when designing space mechanisms?

  Based on the provided sources, it appears that the president did not mention Michael Jackson in any of the documents. The documents are focused on various topics related to space exploration and lubrication, including the use of different types of lubricants in space mechanisms, the challenges of lubricating complex mechanisms in robotic space exploration missions, and the development of new hydrocarbon-based lubricants. Therefore, there is no mention of Michael Jackson in any of these documents.

[{'document_id': 'AMS_1998.pdf_31107727471-051f-4979-8b7b-ff5aafb8233d', 'page': '311', 'source': 'AMS_1998.pdf'}, {'document_id': 'AMS_2020.pdf_2684dd10ab7-7371-4b76-9347-8e22cb98ea41', 'page': '268', 'source': 'AMS_2020.pdf'}, {'document_id': 'AMS_2020.pdf_2655d737dec-85ef-4c92-a748-5606ab9cf2bc', 'page': '265', 'source': 'AMS_2020.pdf'}, {'document_id': 'AMS_2020.pdf_308b4ceecb7-7783-48c3-9975-62fd28ebe114', 'page': '308', 'source': 'AMS_2020.pdf'}]


## Second Prompt

In [27]:
# Generate a response using your chat model
tags=['prompt2_nofilter']
qa_model_obj.update_model(llm=llm,
                          filter_arg=False)

qa_model_obj.query_docs(prompt[1],
                        tags=tags)
ai_response=qa_model_obj.result['answer']
references=qa_model_obj.sources[-1]

display(Markdown(prompt[1]))
display(Markdown(ai_response))
pprint.pprint(references,width=500)

Prompt 2: Can you speak to what failures have occurred when using mineral oil lubricants?

  Based on the provided documents, here are some key takeaways related to lubrication in space mechanisms:
1. Reliability is critical: With the increasing number of space missions and longer mission times, it is essential to have a robust design with high reliability.
2. Long life for components: To improve reliability, it is necessary to have long lives for all components in the design, which requires the use of lubricants that can provide consistent performance over time.
3. Managing thin films: Spacecraft component life often depends on managing thin films in an evolving environment. Oil mobility decreases with film thickness, and new techniques can detect subtle changes in lubricant degradation.
4. Heritage lubricants: PFPE and MAC are heritage lubricants used in space applications due to their outstanding resistance to outgassing. However, they both have benefits and drawbacks, including the formation of iron fluorides in tribocontacts.
5. New techniques for detection: Experimental techniques such as atomic force microscopy (AFM) and X-ray microtomography (XMT) can detect subtle changes in lubric

[{'document_id': 'AMS_2020.pdf_314f564551c-6e39-4ac2-b21c-af1d540e1150', 'page': '314', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2004.pdf_3714ae8b2f2-e157-47ce-8497-e5f92628c864', 'page': '371', 'source': 'AMS_2004.pdf'},
 {'document_id': 'AMS_1997.pdf_289ab65038c-470e-4a9f-b81f-bc837a90ff37', 'page': '289', 'source': 'AMS_1997.pdf'},
 {'document_id': 'AMS_2020.pdf_311a523abb5-56f7-4a2b-974a-8675cab12b01', 'page': '311', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_3113d25b091-9d8f-4d1b-8390-bd85444d3db0', 'page': '311', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_2684dd10ab7-7371-4b76-9347-8e22cb98ea41', 'page': '268', 'source': 'AMS_2020.pdf'}]
